In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
#import time

import random as random
import gmaps
from pprint import pprint

#import pycountry
#from IPython.display import (Image, HTML)
#import logging
#import math
#from scipy import stats

# Functions to find the distance between 2 lat/long coordinates
# Description: https://pypi.org/project/geopy/1.9.1/
from geopy.distance import (distance, great_circle)

# Keys
from api_config import (key_openweathermap, key_gmaps)

# Incorporated citipy to determine city based on latitude and longitude
#from citipy import citipy


In [2]:
# FYI: Function zipcode_from_latlong()
# A function to use reverse geocode lookup to find a
#  postal_code (zipcode) associated with a lat/long coord

def zipcode_from_latlong( a_lat, a_long ):
    baseurl = "https://maps.googleapis.com/maps/api/geocode/json?"
    latlong = f"latlng={a_lat},{a_long}"
    api_key = f"&key={key_gmaps}"
    
    full_url = baseurl + latlong + api_key

    # Perform a reverse geocode loopup to find the zipcode associated with this lat/long coord
    g_response = requests.get(full_url)
    g_json = g_response.json()
    
    # Traverse the results to find a zipcode for this address
    zipcode = None
    for a in r_json['results'][0]['address_components']:
        if 'postal_code' in a['types']:
            zipcode = a['long_name']
            
    # Return the zipcode that was found
    return zipcode

# Data sources for analysis
* Chicago Transit Authority: https://www.transitchicago.com/developers/
    * CTA Transit Stops - General Transit Feed Specification (GTFS): "Data/access-Chicago-CTA-stops.txt"

In [3]:
# Read in the Chicago CTA stop + zipcode info previously cleaned
i_file = "../Data/chicago_cta_stops.csv"

# Read the data into a dataframe
c_cta_stops_df = pd.read_csv(i_file)

# Data types
# Note: 'stop_code' and 'parent_station' both are type 'float64' since these columns may have NaN values.
# (For some reason, NaN results in an error if the data type is 'int64')
c_cta_stops_df.dtypes

# Quick preview of the data
c_cta_stops_df.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,postal_code
0,1,1.0,Jackson & Austin Terminal,"Jackson & Austin Terminal, Northeastbound, Bus...",41.876322,-87.774105,0,NaN,1,60304
1,2,2.0,5900 W Jackson,"5900 W Jackson, Eastbound, Southside of the St...",41.877067,-87.771318,0,NaN,1,60644
2,3,3.0,Jackson & Menard,"Jackson & Menard, Eastbound, Southside of the ...",41.876957,-87.769750,0,NaN,1,60644
3,4,4.0,5700 W Jackson,"5700 W Jackson, Eastbound, Southside of the St...",41.877024,-87.767451,0,NaN,1,60644
4,6,6.0,Jackson & Lotus,"Jackson & Lotus, Eastbound, Southeast Corner",41.876513,-87.761446,0,NaN,1,60644


In [4]:
c_cta_stops_df[ 'postal_code' ].value_counts().head(10)

60619    386
60632    326
60617    312
60621    311
60609    294
60623    292
60641    291
60647    273
60612    272
60620    269
Name: postal_code, dtype: int64

In [5]:
a1 = (39.7612992, -86.1519681)
a2 = (39.762241,  -86.158436 )
dist_d = distance(a1, a2).feet
print (dist_d)

1850.4323227151917


In [6]:
tempDataList = [(39.7612992, -86.1519681), 
                (39.762241,  -86.158436 ), 
                (39.7622292, -86.1578917)]

def closest(data, y):
    retval = min( data, key=lambda z: distance( z, y ).feet )
    return retval

r = (39.7622290, -86.1519750)

closest(tempDataList, r)

(39.7612992, -86.1519681)

In [7]:
# Function to find the CTA stop closest to the reference point provided
# r: reference point as a tuple (lat, long)
# stop_coords: a list of tuples with ('stop_lat', 'stop_lon')
#               generated from the dataframe containing CTA stops

def closest_stop(stop_coords, r):
    # Find the lat/long tuple closest to the reference point provided
    close_point = min( stop_coords, key=lambda z: distance( z, r ).feet )
    
    # Get the index of this closest point
    # (Note, if there are dups in the list just return the first index)
    retval = stop_coords.index( close_point )
    return retval

# Generate the list of CTA stop coordinates
stop_coords = list( zip( c_cta_stops_df['stop_lat'], c_cta_stops_df['stop_lon'] ) )

# A reference point for which we want to find the index of the closest CTA stop
refpt = (41.87659462, -87.75461525)

# Index of the closest CTA stop
closest_stop_index = closest_stop( stop_coords, refpt)

c_cta_stops_df.loc[closest_stop_index,'postal_code']

60644

# Group and Merge for Analysis
## Comparison of ratings and transit stop density on per-zipcode basis

In [8]:
# Read in zip code data (which will provide lat/long for each zip code)
i_file = "../Data/zip_code_database.csv"
zip_info_df = pd.read_csv(i_file)
zip_info_df.head()

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population_2015
0,501,UNIQUE,0,Holtsville,NaN,I R S Service Center,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,562
1,544,UNIQUE,0,Holtsville,NaN,Irs Service Center,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,0
2,601,STANDARD,0,Adjuntas,NaN,"Colinas Del Gigante, Jard De Adjuntas, Urb San...",PR,Adjuntas Municipio,America/Puerto_Rico,"787,939",NaN,US,18.16,-66.72,0
3,602,STANDARD,0,Aguada,NaN,"Alts De Aguada, Bo Guaniquilla, Comunidad Las ...",PR,Aguada Municipio,America/Puerto_Rico,"787,939",NaN,US,18.38,-67.18,0
4,603,STANDARD,0,Aguadilla,Ramey,"Bda Caban, Bda Esteves, Bo Borinquen, Bo Ceiba...",PR,Aguadilla Municipio,America/Puerto_Rico,787,NaN,US,18.43,-67.15,0


In [9]:
# The zip_info_df file about has lat/long coordinates with insufficient precision
#  if a distance search has to be performed -- get a different zip code source
# Reference: https://gist.github.com/erichurst/7882666
i_file = "../Raw Data/US_Zip_Codes_from_2013_Government_Data.csv"
zip_latlong_df = pd.read_csv(i_file)
zip_latlong_df.head()

,ZIP,LAT,LNG
0,601,18.180555,-66.749961
1,602,18.361945,-67.175597
2,603,18.455183,-67.119887
3,606,18.158345,-66.932911
4,610,18.295366,-67.125135


In [10]:
zip_latlong_df['ZIP'].count()

33144

In [11]:
# The zip_latlong_df has more zip codes that are needed
# - limit only to Chicago area to speed processing later
# Reference: https://www.zip-codes.com/city/il-chicago.asp#zipcodes
zip_c_only_df = (zip_latlong_df.loc[ zip_latlong_df['ZIP'] > 60000 ]).loc[ zip_latlong_df['ZIP'] < 61000 ]
zip_c_only_df.reset_index(drop=True, inplace=True)

In [12]:
zip_c_only_df.count()

ZIP    393
LAT    393
LNG    393
dtype: int64

In [13]:
zip_c_only_df.set_index('ZIP', inplace=True)

In [14]:
zip_c_only_df.head()

,LAT,LNG
ZIP,,
60002,42.471741,-88.084493
60004,42.112780,-87.979542
60005,42.064490,-87.985462
60007,42.008600,-87.997340
60008,42.069786,-88.016221


In [15]:
zip_c_only_df.head()

,LAT,LNG
ZIP,,
60002,42.471741,-88.084493
60004,42.112780,-87.979542
60005,42.064490,-87.985462
60007,42.008600,-87.997340
60008,42.069786,-88.016221


In [16]:
# Much better: only 393 zip codes in Chicago
zip_c_only_df.count()

LAT    393
LNG    393
dtype: int64

In [17]:
# Read in yelp data will will provide restaurant ratings, cuisine, lat/long, etc.
i_file = "../Data/Yelp_Restaurants_Chicago.csv"
rest_zip_df = pd.read_csv(i_file)
rest_zip_df.head()

,zip,city,state,name,price,rating,review_count,type,latitude,longitude
0,60601,Chicago,IL,Wildberry Pancakes and Cafe,$$,4.5,5862,American (New),41.884668,-87.622880
1,60603,Chicago,IL,The Gage,$$,4.0,2632,American (New),41.881048,-87.624533
2,60603,Chicago,IL,Cindy's,$$,4.0,1387,American (New),41.881689,-87.625006
3,60603,Chicago,IL,The Marq,$$,4.0,509,American (New),41.879619,-87.629925
4,60603,Chicago,IL,Cherry Circle Room,$$$,4.5,330,American (New),41.881664,-87.624969


In [18]:
# Chicago CTA stops data
c_cta_stops_df.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,postal_code
0,1,1.0,Jackson & Austin Terminal,"Jackson & Austin Terminal, Northeastbound, Bus...",41.876322,-87.774105,0,NaN,1,60304
1,2,2.0,5900 W Jackson,"5900 W Jackson, Eastbound, Southside of the St...",41.877067,-87.771318,0,NaN,1,60644
2,3,3.0,Jackson & Menard,"Jackson & Menard, Eastbound, Southside of the ...",41.876957,-87.769750,0,NaN,1,60644
3,4,4.0,5700 W Jackson,"5700 W Jackson, Eastbound, Southside of the St...",41.877024,-87.767451,0,NaN,1,60644
4,6,6.0,Jackson & Lotus,"Jackson & Lotus, Eastbound, Southeast Corner",41.876513,-87.761446,0,NaN,1,60644


# Useful distance related functions, for later... maybe...

In [19]:
# Function to find the a (lat, long) coord that is closest to a reference point
#  and then return the index of the coord in the provided list of coords
# Note: If the coordinates are duplicated in the list of coordinates,
#        then the index of the first coordinate is returned
# r: reference point as a tuple (lat, long)
# stop_coords: a list of tuples with ('stop_lat', 'stop_lon')
#               generated from the dataframe containing CTA stops

def closest_coord(coords, r):
    # Find the lat/long tuple closest to the reference point provided
    close_point = min( coords, key=lambda z: distance( z, r ).feet )
    
    # Get the index of this closest point in the list of coordinates
    # (Note, if there are dups in the list just return the first index)
    retval = coords.index( close_point )
    return retval

In [20]:
# USE LATER:
# Generate the list of CTA stop coordinates,
# which can be used to lookup the CTA stop  that is closest to a specific restaurant
# stop_coords = list( zip( c_cta_stops_df['stop_lat'], c_cta_stops_df['stop_lon'] ) )

In [21]:
# Use the index of the closest CTA stop to extract specific info out of the CTA stop dataframe
# closest_stop_index = closest_coord( stop_coords, refpt)
# c_cta_stops_df['stop_name'][closest_stop_index]

## Calculate the total number of CTA stops per zip code

In [22]:
# Obtain the count of each postal_code (zipcode) from the Chicago CTA stop info
c_cta_stop_count = c_cta_stops_df['postal_code'].value_counts()

# Sort the Series by its index (i.e., zipcode)
c_cta_stop_count.sort_index(inplace=True)

# Rename the series to 'Total CTA Stops'
c_cta_stop_count.rename("Total CTA Stops", inplace=True)

c_cta_stop_count.head()

60018     1
60029    15
60053     1
60068    10
60076    27
Name: Total CTA Stops, dtype: int64

## Aggregrate the restaurant metrics by zipcode

In [23]:
# Restarant data from Yelp
rest_zip_df.head()

,zip,city,state,name,price,rating,review_count,type,latitude,longitude
0,60601,Chicago,IL,Wildberry Pancakes and Cafe,$$,4.5,5862,American (New),41.884668,-87.622880
1,60603,Chicago,IL,The Gage,$$,4.0,2632,American (New),41.881048,-87.624533
2,60603,Chicago,IL,Cindy's,$$,4.0,1387,American (New),41.881689,-87.625006
3,60603,Chicago,IL,The Marq,$$,4.0,509,American (New),41.879619,-87.629925
4,60603,Chicago,IL,Cherry Circle Room,$$$,4.5,330,American (New),41.881664,-87.624969


In [24]:
# Convert the price from '$' format to numerical value
rest_zip_df['price_num'] = rest_zip_df['price'].apply(len)
rest_zip_df.head()

,zip,city,state,name,price,rating,review_count,type,latitude,longitude,price_num
0,60601,Chicago,IL,Wildberry Pancakes and Cafe,$$,4.5,5862,American (New),41.884668,-87.622880,2
1,60603,Chicago,IL,The Gage,$$,4.0,2632,American (New),41.881048,-87.624533,2
2,60603,Chicago,IL,Cindy's,$$,4.0,1387,American (New),41.881689,-87.625006,2
3,60603,Chicago,IL,The Marq,$$,4.0,509,American (New),41.879619,-87.629925,2
4,60603,Chicago,IL,Cherry Circle Room,$$$,4.5,330,American (New),41.881664,-87.624969,3


In [25]:
# Aggregation functions to apply to the various columns
agg_funcs_dict = {'name':'count',
                  'rating':'mean',
                  'review_count':['median','mean','sum'],
                  'price_num':'mean'
                 }

In [26]:
rest_z_g = rest_zip_df.groupby('zip')
rest_z_g_df = rest_z_g.agg(agg_funcs_dict)
rest_z_g_df.head()

name    rating review_count                    price_num
      count      mean       median        mean    sum      mean
zip                                                            
60601    14  3.500000        397.5  823.928571  11535  1.928571
60602    11  3.863636        390.0  371.454545   4086  1.454545
60603    25  3.680000        509.0  568.200000  14205  2.040000
60604     2  4.000000        406.5  406.500000    813  1.500000
60605    32  3.953125        334.0  464.718750  14871  1.875000

In [27]:
# Start building the merged dataframe
merged_rest_df = pd.DataFrame()

In [28]:
merged_rest_df['Total Restaurants'] = rest_z_g_df['name']['count']
merged_rest_df['Avg Rating'] = rest_z_g_df['rating']['mean']
merged_rest_df['Total Reviews'] = rest_z_g_df['review_count']['sum']
merged_rest_df['Median Reviews'] = rest_z_g_df['review_count']['median']
merged_rest_df['Avg Reviews'] = rest_z_g_df['review_count']['mean']
merged_rest_df['Avg Price (# of $)'] = rest_z_g_df['price_num']['mean']
merged_rest_df.head()

,Total Restaurants,Avg Rating,Total Reviews,Median Reviews,Avg Reviews,Avg Price (# of $)
zip,,,,,,
60601,14,3.500000,11535,397.5,823.928571,1.928571
60602,11,3.863636,4086,390.0,371.454545,1.454545
60603,25,3.680000,14205,509.0,568.200000,2.040000
60604,2,4.000000,813,406.5,406.500000,1.500000
60605,32,3.953125,14871,334.0,464.718750,1.875000


In [29]:
len(c_cta_stop_count)

88

In [30]:
len(merged_rest_df)

57

In [31]:
# Merge the CTA stop data into the Restaurant data
# Note: Use .to_frame() to convert the CTA Stop count series into a dataframe for the merge
merged_rest_df = pd.merge(merged_rest_df, c_cta_stop_count.to_frame(),
         how='left', left_on='zip', right_index=True)
merged_rest_df.head()

,Total Restaurants,Avg Rating,Total Reviews,Median Reviews,Avg Reviews,Avg Price (# of $),Total CTA Stops
zip,,,,,,,
60601,14,3.500000,11535,397.5,823.928571,1.928571,52.0
60602,11,3.863636,4086,390.0,371.454545,1.454545,39.0
60603,25,3.680000,14205,509.0,568.200000,2.040000,11.0
60604,2,4.000000,813,406.5,406.500000,1.500000,58.0
60605,32,3.953125,14871,334.0,464.718750,1.875000,60.0


## Add in the (lat, long) coordinates for each zip code

In [32]:
zip_c_only_df.head()

,LAT,LNG
ZIP,,
60002,42.471741,-88.084493
60004,42.112780,-87.979542
60005,42.064490,-87.985462
60007,42.008600,-87.997340
60008,42.069786,-88.016221


In [33]:
# Merge the zip code lat/long data into the merged Restaurant dataframe
merged_rest_df = pd.merge(merged_rest_df, zip_c_only_df,
         how='left', left_index=True, right_index=True)

# Rename the lat/long columns
merged_rest_df = merged_rest_df.rename( columns={'LAT': 'Latitude', 'LNG': 'Longitude'})
merged_rest_df.head()

,Total Restaurants,Avg Rating,Total Reviews,Median Reviews,Avg Reviews,Avg Price (# of $),Total CTA Stops,Latitude,Longitude
zip,,,,,,,,,
60601,14,3.500000,11535,397.5,823.928571,1.928571,52.0,41.885310,-87.622116
60602,11,3.863636,4086,390.0,371.454545,1.454545,39.0,41.883073,-87.629149
60603,25,3.680000,14205,509.0,568.200000,2.040000,11.0,41.880188,-87.625509
60604,2,4.000000,813,406.5,406.500000,1.500000,58.0,41.878095,-87.628461
60605,32,3.953125,14871,334.0,464.718750,1.875000,60.0,41.867566,-87.617228


In [35]:
# Save the merged dataframe for general use
o_file = "../Data/merged_restaurants_and_CTA_stops.csv"
merged_rest_df.to_csv(o_file, index=True)